In [1]:
%cd '/gdrive/MyDrive/2021-1-Mentor(CNN)/2021-1-Classification_4classes/code'

/gdrive/MyDrive/2021-1-Mentor(CNN)/2021-1-Classification_4classes/code


In [2]:
import os, math, time
import numpy as np
import matplotlib.pyplot as plt



# seed for reproducibility
import random 
import tensorflow as tf
from keras import backend as K
seed_value=101;
os.environ['PYTHONHASHSEED']=str(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)


import keras
from keras import models, layers
from tensorflow.keras.callbacks import Callback
from IPython.display import clear_output
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics

from sklearn.utils import class_weight
from keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

In [3]:
# parameters
NUM_TARGET_LABELS = 4

file_path = './'

fname_image = '../data/data_image.npy'
fname_labels = '../data/labels.npy'
label_names = ['com','cod','ca','st','ua','au','cy','br','others']

exp_name = 'ResNet152_origin'
result_path = f'../result/{exp_name}'

In [4]:
######################## IF WINDOWS,
gpu = tf.config.experimental.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(gpu[0], True) 
except RuntimeError as e:
    print(e) 

In [5]:
####################################################
#######  EXECUTE ONLY IF USING GOOGLE COLAB ########
from google.colab import drive
drive.mount('/gdrive')

file_path = '/gdrive/MyDrive/2021-1-Mentor(CNN)/2021-1-Classification_4classes/code'
name_image = os.path.join(file_path, fname_image)
name_labels = os.path.join(file_path, fname_labels)
####################################################

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [6]:
### set data and labels
# load data and labels
data = np.load(fname_image)
labels = np.load(fname_labels)

# set unique labels
label_unique = np.array([list(x) for x in set(tuple(x) for x in labels)])
label_unique_indices = np.array([list(np.where((labels == val).all(axis=1)))[0] for val in label_unique])
label_unique_counts = np.array([len(x) for x in label_unique_indices])

# sort by frequency
temp_ind = label_unique_counts.argsort()[::-1]
label_class_info = label_unique[temp_ind]
label_class_indices = label_unique_indices[temp_ind]
label_class_counts = label_unique_counts[temp_ind]

# final label and info
label_final = labels[:,0]
for label_i in range(len(label_unique_counts)):
    label_final[label_class_indices[label_i]] = label_i

label_info = [label_class_info[x.astype(int)] for x in label_final]


# cut data and labels
target_ind = np.array(label_final) >= NUM_TARGET_LABELS
label_other = np.array(label_final)[target_ind]
data_other = data[target_ind] / 255.0

target_ind = np.array(label_final) < NUM_TARGET_LABELS
label_final = np.array(label_final)[target_ind]
data_final = data[target_ind] / 255.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [7]:
### model and function definition



# model class
class Network():
  start_time = 0

  num_output_label = 4

  def __init__(self, num_output_label=4):
    self.num_output_label = num_output_label

  def create(self):
    self.model = models.Sequential()
    self.model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(160, 160, 3)))
    self.model.add(layers.MaxPooling2D((2, 2)))
    self.model.add(layers.Dropout(0.25))

    self.model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    self.model.add(layers.MaxPooling2D((2, 2)))
    self.model.add(layers.Dropout(0.25))

    self.model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    self.model.add(layers.MaxPooling2D((2, 2)))
    self.model.add(layers.Dropout(0.25))

    self.model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    self.model.add(layers.MaxPooling2D((2, 2)))
    self.model.add(layers.Dropout(0.25))

    self.model.add(layers.Flatten())
    self.model.add(layers.Dense(256, activation='relu'))
    self.model.add(layers.Dropout(0.3))

    self.model.add(layers.Dense(self.num_output_label, activation='softmax'))

    self.model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


#    def create(self):
#        self.model=tf.keras.application.ResNet152(
#            weights='imagenet', include_top=False, input_shape=(160, 160, 3)
#        )


# training history callback class
class TrainingHistoryCallback(Callback):

  def __init__(self, fold_i, x_test, y_test, save_path, visualize=True):
    self.fold_i = fold_i
    self.x_test = x_test
    self.y_test = y_test
    self.save_path = save_path
    self.visualize = visualize

  def on_train_begin(self, logs={}):
    self.update_freq = 10
    self.curr_epoch = 0
    self.x = []

    self.train_loss = []
    self.train_acc = []
    self.val_loss = []
    self.val_acc = []

    self.fig = plt.figure()

    self.logs = []
    self.y_pred = []

    self.best_val_loss = 1000000
    self.best_acc = 0
    self.best_f1 = 0
    
    self.best_val_loss_epoch = None
    self.best_acc_epoch = None
    self.best_f1_epoch = None
    

  def on_epoch_end(self, epoch, logs={}):
    self.curr_epoch += 1
    self.x.append(self.curr_epoch)
    self.train_loss.append(logs.get('loss'))
    self.train_acc.append(logs.get('accuracy'))
    self.val_loss.append(logs.get('val_loss'))
    self.val_acc.append(logs.get('val_accuracy'))
    self.logs.append(logs)
    self.y_pred.append(self.model.predict(self.x_test))

    # check whether model has the best validation loss
    if logs.get('val_loss') < self.best_val_loss:
      self.best_val_loss = logs.get('val_loss')
      self.best_val_loss_epoch = self.curr_epoch
      self.model.save(os.path.join(self.save_path,'model_best_loss.hdf5'),overwrite=True)
        
    # check whether model has the best validation accuracy
    if logs.get('val_accuracy') > self.best_acc:
      self.best_acc = logs.get('val_accuracy')
      self.best_acc_epoch = self.curr_epoch
      self.model.save(os.path.join(self.save_path,'model_best_acc.hdf5'),overwrite=True)
    
    # check whether model has the best mean f1 score
    _, _, fscore, _ = metrics.precision_recall_fscore_support(np.argmax(self.y_test, axis=1),
                                                              np.argmax(self.model.predict(self.x_test),axis=1))
    if np.mean(fscore) > self.best_f1:
      self.best_f1 = np.mean(fscore)
      self.best_f1_epoch = self.curr_epoch
      self.model.save(os.path.join(self.save_path,'model_best_f1.hdf5'),overwrite=True)
        
    # visualization
    if (self.visualize and self.curr_epoch % self.update_freq == 0):
      clear_output(wait=True)

      plt.figure(figsize=(12,5))
      plt.subplot(121)
      plt.plot(self.x[5:], self.train_loss[5:], label="train_loss")
      plt.plot(self.x[5:], self.val_loss[5:], label="val_loss")
      plt.legend()

      plt.subplot(122)
      plt.plot(self.x, self.train_acc, label="train_acc")
      plt.plot(self.x, self.val_acc, label="val_acc")
      plt.legend()
      plt.show()

      print("fold = ",self.fold_i, "epoch =",self.curr_epoch)
      print("\ttrain_loss = ", self.train_loss[-1], ", val_loss = ", self.val_loss[-1])
      print("\t\ttrain_acc = ", self.train_acc[-1], ", val_acc = ", self.val_acc[-1])


In [8]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications import resnet
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout
from tensorflow.keras.models import Model

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers

from tensorflow.keras.applications import ResNet152
# 모델 불러오기
conv_layers = ResNet152(weights='imagenet', include_top=False, input_shape=(160, 160, 3), pooling='max')
#conv_layers.summary()

# Convolution Layer를 학습되지 않도록 고정 
#for layer in conv_layers.layers:
#    layer.trainable = False

# 새로운 모델 생성하기
model = models.Sequential()

# VGG16모델의 Convolution Layer를 추가
model.add(conv_layers)
 
# 모델의 Fully Connected 부분을 재구성
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))

#model.summary() -> built first?

In [9]:
#0609 keras.utils.to_categorical 에러
#AttributeError: module 'keras.utils' has no attribute 'to_categorical'

from tensorflow import keras

### train and evaluate model
# one-hot encoding
label_final_onehot = keras.utils.to_categorical(label_final)

# kfold
kfold_divider = StratifiedKFold(n_splits = 10, shuffle=True)

# initialize
os.mkdir(os.path.join(result_path))


# main
for fold_i, (ind_train,ind_test) in enumerate(kfold_divider.split(data_final, label_final)):
  # set path to save
  result_path_current = f'{result_path}/{fold_i+1}'
  os.mkdir(os.path.join(file_path,result_path_current))

  # shuffle
  x_train, x_test = data_final[ind_train], data_final[ind_test]
  y_train, y_test = label_final_onehot[ind_train], label_final_onehot[ind_test]



  # data augmentation
  data_generator = ImageDataGenerator(rotation_range=90,  
                                      zoom_range=[0.5,1.0],
                                      horizontal_flip=True,
                                      vertical_flip=True,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      fill_mode='wrap')



  # class weighting
  label_int = np.argmax(label_final_onehot, axis=1)
  class_weights = class_weight.compute_class_weight('balanced',  np.unique(label_int), label_int)
  class_weights_dict = dict(enumerate(class_weights))



  # training initialization
  # network = Network(num_output_label=NUM_TARGET_LABELS)
  #network = model
  
  history_callback = TrainingHistoryCallback(fold_i+1, x_test, y_test, result_path_current, visualize=True)
  #checkpoint_callback = keras.callbacks.ModelCheckpoint(os.path.join(result_path_current,'model.hdf5'), verbose=0, save_best_only=True)
  earlystop_callback = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=150)

  start_time = time.time()

  # training
  #network.create()
  #network.summary()
  network = model


  batch_size=64
  epochs=1000


  #모델 컴파일

  network.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])
  #모델학습 
  training_history = network.fit(
      data_generator.flow(x_train, y_train, batch_size=batch_size),
      validation_data=(x_test,y_test),
      steps_per_epoch=x_train.shape[0] // batch_size,
      epochs=epochs, verbose=1,
      class_weight=class_weights_dict, 
      callbacks = [history_callback, earlystop_callback]    
  )
  #print("elapsed : {}".format(time.time() - start_time))
  
  network.summary()

  # save result
  elapsed_time = np.array(time.time() - start_time)

  x_test
  x_other = data_other
  y_test = np.argmax(y_test, axis=1)
  ind_test = np.array(ind_test)

  epoch_list = np.array(history_callback.x)
  train_loss = np.array(history_callback.train_loss)
  train_acc = np.array(history_callback.train_acc)
  val_loss = np.array(history_callback.val_loss)
  val_acc = np.array(history_callback.val_acc)

  pred_history = np.array(history_callback.y_pred)


  best_val_loss_epoch = history_callback.best_val_loss_epoch
  best_acc_epoch = history_callback.best_acc_epoch
  best_f1_epoch = history_callback.best_f1_epoch

  np.savez(os.path.join(result_path_current,'mat.npz'),
          elapsed_time=elapsed_time,
          x_test=x_test, y_test=y_test, ind_test=ind_test,
          epoch_list=epoch_list,
          train_loss=train_loss, train_acc=train_acc,
          val_loss=val_loss, val_acc=val_acc,
          pred_history=pred_history,
          x_other=x_other,
          best_val_loss_epoch=best_val_loss_epoch, best_acc_epoch=best_acc_epoch, best_f1_epoch=best_f1_epoch
          )


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/1000


ResourceExhaustedError: ignored

<Figure size 432x288 with 0 Axes>

In [ ]:
### evaluation
y_true = np.argmax(y_test, axis=1)
y_pred = np.argmax(history_callback.best_model.predict(x_test),axis=1)
accuracy = metrics.accuracy_score(y_true,y_pred)
precision, sensitivity, fscore, support = metrics.precision_recall_fscore_support(y_true, y_pred)
auc1 = metrics.roc_auc_score(y_true, history_callback.best_model.predict(x_test), multi_class='ovo')
auc2 = metrics.roc_auc_score(y_true, history_callback.best_model.predict(x_test), multi_class='ovr')
conf_mat = confusion_matrix(y_true,y_pred)

print(accuracy)
print(precision)
print(recall)
print(fscore)
print(support)
print(auc1)
print(auc2)
print(conf_mat)